In [1]:
import os
import pandas as pd
import numpy as np
import re

In [44]:
df_origin = pd.read_excel('../Data/Preprocessed/탈모톡톡_19~23세탈모.xlsx')
df_origin = df_origin[df_origin['keyword'] != '[]']
df_origin['view'] = df_origin['view'].apply(lambda x : int(re.sub('[^0-9]', '', x)))
df_origin['keyword'] = df_origin['keyword'].apply(lambda x : eval(x))

columnList = df_origin['category'].unique().tolist()
for removeColumn in ['[미용실]', '[]', '[학교생활]', '[이성고민]', '[연애고민]']:
    columnList.remove(removeColumn)

df_origin = df_origin[df_origin['category'].isin(columnList)]


C:\Users\WESLEYQUEST\AppData\Local\Temp\ipykernel_20916\3226866085.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  df_origin = pd.read_excel('../Data/Preprocessed/탈모톡톡_19~23세탈모.xlsx')


In [45]:
def painPoint(x):
    x = x.sort_values(by = ['view', 'date'], ascending = False)
    return x.iloc[:3]

In [46]:
df = df_origin.copy()
result = df.groupby(['propTopic', 'cluster']).apply(lambda x : painPoint(x))
result.to_excel('../Result/PainPoint/19~23세탈모.xlsx', index = False)

In [47]:
df[(df['propTopic'] == 10) & (df['keyword'].apply(lambda x : '사진' in x ))].sort_values(by = ['view'], ascending = False)['content'].iloc[3]

'집안이 머리 가늘고 숱이 적지만  탈모는 없고 나도 원래 머리숱 없다는 소리 듣고  자랐지만 어느정도 없는지 확인을 안해와서  잘모르겠는데  커서 보니 정수리쪽에ㅠ머리숱이 많이 없는거 같고...  사진 찍어보니깐 어떤건 괜찮아 보이고 어떤건 엄청 심해보이고... 괜히 속상하네  생활습관이 개판이고  스트레스도 많이 받는 편이긴 한데  개선하면 나아질까?  그리고 머리에 물묻으면 원래 두피 많이 보여?'

In [86]:
# category별 propTopic 게시글 수 비율
df = df_origin.copy()
tmp = df.groupby(['category', 'propTopic'])['href'].count().reset_index()
tmp['href'] = tmp['href'].apply(lambda x : round(x/tmp['href'].sum() * 100, 2))
result = tmp.pivot_table(index = ['category'], columns = ['propTopic'], values = ['href']).fillna(0)

result['sum'] = np.nan
for i in range(len(result)):
    result.iloc[i, -1] = result.iloc[i].sum()

for i, column in enumerate(result.columns):
    result.loc['합계', column] = result.iloc[:, i].sum()

result01 = result.copy()

In [108]:
result01

href                                                          \
propTopic      0      1     2     3     4      5     6     7     8     9   
category                                                                   
[19세탈모]     1.84   2.25  0.62  0.51  0.46   2.88  0.11  1.06  0.30  0.57   
[20세탈모]     1.92   2.35  0.73  0.68  0.80   3.53  0.25  1.36  0.47  0.67   
[21세탈모]     1.52   1.93  0.81  0.67  0.50   3.23  0.33  1.02  0.45  0.54   
[22세탈모]     1.30   2.00  0.76  0.67  0.51   3.05  0.43  0.85  0.48  0.77   
[23세탈모]     5.73   7.19  2.75  2.88  1.69  12.38  1.68  3.62  1.31  2.98   
[군대문제]      0.16   0.39  0.28  0.13  0.02   0.29  0.15  0.01  0.00  0.11   
합계         12.47  16.11  5.95  5.54  3.98  25.36  2.95  7.92  3.01  5.64   

                    sum  
propTopic     10         
category                 
[19세탈모]     2.24  12.84  
[20세탈모]     1.67  14.43  
[21세탈모]     1.32  12.32  
[22세탈모]     1.15  11.97  
[23세탈모]     4.61  46.82  
[군대문제]      0.07   1.61  
합계         11.06  99.99

In [87]:
# category별 propTopic 게시글 평균 조회수
df = df_origin.copy()
tmp = df.groupby(['category', 'propTopic'])['view'].mean().reset_index()
tmp['view'] = tmp['view'].apply(lambda x : round(x))
result = tmp.pivot_table(index = ['category'], columns = ['propTopic'], values = ['view']).fillna(0)

result['전체'] = np.nan
for i in range(len(result)):
    result.iloc[i, -1] = df[df['category'] == result.index[i]]['view'].mean()
for i, column in enumerate(result.columns[:-1]):
    result.loc['평균', column] = df[df['propTopic'] == i]['view'].mean()

result = result.fillna(0)
result02 = result.astype(int)

In [88]:
result02

view                                                              \
propTopic     0     1     2     3     4     5     6     7     8     9    10   
category                                                                      
[19세탈모]    1787  1660  2687  1291  2962  1684   785  1526  2303  1446  2168   
[20세탈모]    1339  1823  2673   947  1567  1776  1543  1824  2549  1543  1928   
[21세탈모]    1609  1621  2108  1235  2537  1717  1563  1890  2515  1062  2604   
[22세탈모]    1393  1626  1856  1195  2197  1616  1343  1842  2514  1429  1752   
[23세탈모]    1460  1434  2400  1080  2048  1610  1288  1767  2752  1360  1778   
[군대문제]     1840  2265  3115  2788   854  1998  1496  2913     0  1334  2171   
평균         1505  1588  2387  1154  2129  1660  1340  1769  2602  1370  1977   

             전체  
propTopic        
category         
[19세탈모]    1840  
[20세탈모]    1752  
[21세탈모]    1826  
[22세탈모]    1653  
[23세탈모]    1627  
[군대문제]     2208  
평균            0

In [113]:
df_origin.head(2)

,title,category,date,writer,rank,view,href,content,imageList,titleContent,preprocessedSentence,keyword,sentimentResult,propTopic,propTopicRatio,topicsRatio,cluster
0,친가가 탈모 이력이 있으면 100프로겠지??,[22세탈모],2018-03-04,아빠는왜하필,2,603,https://daedamo.com/1923club/72131?page=136&ov...,하 원래도 머리숱이 적은 편인데 ㅋㅋㅋ 점점 머리감을때 두피가 만져지고 앞머리도 ...,[],친가가 탈모 이력이 있으면 100프로겠지?? 하 원래도 머리숱이 적은 편인데 ㅋㅋㅋ...,친가가 탈모 이력이 있으면 100프로겠지?? 하 원래도 머리숱이 적은 편인데 점...,"[가가, 탈모, 이력, 있다, 프로, 원래, 머리숱, 적, 머리, 감다, 때, 두피...",NaN,0,0.3954,"[(0, 0.3953887), (1, 0.069146655), (2, 0.05387...",5
1,kelopesia 가격 나옴,[22세탈모],2017-05-19,신2이,19,1015,https://daedamo.com/1923club/54677?page=185&ov...,ReBoost international sales started today and ...,[],kelopesia 가격 나옴 ReBoost international sales st...,kelopesia 가격 나옴 ReBoost international sales st...,"[가격, 나오다, 달러]",NaN,3,0.4277,"[(0, 0.36588243), (1, 0.02293169), (2, 0.02293...",3


In [120]:
# 19~23세 탈모 게시글 토픽 및 월별 작성 게시글 수

df = df_origin.copy()
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
result = df.groupby(['year', 'month', 'propTopic'])['href'].count().reset_index().groupby(['month', 'propTopic'])['href'].mean().reset_index().pivot_table(index = ['month'], columns = ['propTopic'], values = ['href'])
result = result.apply(lambda x : round(x, 2))
result

href                                                            \
propTopic     0      1     2     3     4      5     6      7     8      9    
month                                                                        
1          17.00  16.10  7.22  6.30  5.60  25.20  5.40   6.80  3.00   7.00   
2          11.30  14.90  5.30  5.50  3.80  25.70  2.80   5.60  3.25   6.75   
3          12.10  17.60  5.90  7.50  3.44  24.30  3.44   6.10  4.71   6.11   
4          11.90  16.40  6.40  5.33  6.44  29.10  3.70   6.70  2.70   7.00   
5           9.70  13.90  6.10  5.89  2.89  22.30  3.60   7.10  3.89   6.00   
6          11.20  16.40  5.90  5.60  4.10  26.00  2.67   8.40  3.30   6.11   
7          15.50  17.60  6.20  7.11  3.90  28.90  4.50  10.80  3.30   7.70   
8          15.20  21.80  8.11  7.40  4.30  27.90  3.33  14.78  4.33   6.30   
9          15.22  20.33  6.89  7.38  6.11  32.67  2.86  11.56  3.75   5.75   
10         17.89  17.56  6.88  8.12  5.22  34.56  3.38  12.11  3.88  10.00   
11         15.22  20.78  8.11  7.25  3.67  30.67  3.00   9.89  3.78   8.86   
12         10.60  16.80  7.00  5.88  4.33  24.00  3.12   5.89  4.00   5.40   

                  
propTopic     10  
month             
1          12.50  
2          11.20  
3          10.30  
4          11.30  
5           9.70  
6          12.30  
7          13.20  
8          13.50  
9          14.44  
10         11.56  
11         12.33  
12         11.80